In [62]:
import torch
from torchsummary import summary
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.functional as F
from sentence_transformers import SentenceTransformer
from tqdm import tqdm



import pandas as pd
import numpy as np
import os

# StarWars



In [63]:
base_dir = "../data/raw/StarWarsEpisodes"

In [64]:
folder_ep4 = os.path.join(base_dir,"SW_EpisodeIV.txt")
folder_ep5 = os.path.join(base_dir,"SW_EpisodeV.txt")
folder_ep6 = os.path.join(base_dir,"SW_EpisodeVI.txt")

In [65]:
df_ep4 = pd.read_csv(folder_ep4, sep =' ', header=0, escapechar='\\')
df_ep5 = pd.read_csv(folder_ep5, sep =' ', header=0, escapechar='\\')
df_ep6 = pd.read_csv(folder_ep6, sep =' ', header=0, escapechar='\\')


In [66]:
df_ep4


,character,dialogue
1,THREEPIO,Did you hear that? They've shut down the main...
2,THREEPIO,We're doomed!
3,THREEPIO,There'll be no escape for the Princess this time.
4,THREEPIO,What's that?
5,THREEPIO,I should have known better than to trust the l...
...,...,...
1006,LUKE,"Oh, no!"
1007,THREEPIO,"Oh, my! Artoo! Can you hear me? Say somethi..."
1008,TECHNICIAN,We'll get to work on him right away.
1009,THREEPIO,"You must repair him! Sir, if any of my circui..."


In [67]:
Y = pd.concat([df_ep4['character'],df_ep5['character'],df_ep6['character']]).tolist()
X = pd.concat([df_ep4['dialogue'],df_ep5['dialogue'],df_ep6['dialogue']]).tolist()

In [68]:
char2ind = {i:j for i,j in zip(Y,range(len(Y)))}
ind2char = {j:i for i,j in zip(Y,range(len(Y)))}

In [69]:
# char_names = movie_lines.iloc[:,0]
# movie_names = movie_lines.iloc[:,1]
# char_names = np.unique(list(set(char_names.values)))
# movie_names = np.unique(list(set(movie_names.values)))

In [70]:
class CustomStarWarsDataset(Dataset):
    def __init__(self, X, Y,transform=None, target_transform=None):
        self.X = X
        self.Y =Y
        self.sentence_model = SentenceTransformer('bert-base-nli-mean-tokens')
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        data_point = self.X[idx]
        label_point = char2ind[self.Y[idx]]
        # print("data_point is:",data_point)
        # print("label_point is:",label_point)
        sentence_encoded = self.sentence_model.encode(data_point)
        # 768 
        # print(sentence_encoded,label_point)
        return sentence_encoded, label_point

In [79]:
class BertSentenceClassifier(nn.Module):
    def __init__(self):
        super(BertSentenceClassifier, self).__init__()

        self.lin1 = nn.Linear(768,768)
        self.lin2 = nn.Linear(768, 768//2)
        self.lin3 = nn.Linear(768//2, len(Y))



    def forward(self, data):
        x = nn.functional.relu(self.lin1(data))
        x = nn.functional.relu(self.lin2(x))
        x = self.lin3(x)

        x = nn.functional.softmax(x,dim = 1)

        
        return x

In [72]:
train_dataset = CustomStarWarsDataset(X,Y)
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True,drop_last=True)

In [76]:
model = BertSentenceClassifier()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-2)

In [77]:
device = "mps" if torch.backends.mps.is_available() else "cpu"


In [78]:

num_epochs = 3
print(device)
model = model.to(device)

for epoch in range(num_epochs):
    loop = tqdm(train_loader, total=len(train_loader))

    for text, author_labels in loop:  # Assuming data_loader is set up to provide batches of data
        data = text.to(device)
        author_labels = author_labels.to(device)
        optimizer.zero_grad()

        data = model(data)
        # print(data.shape)
        # print("data and author labels ::::: ",data,author_labels)
        loss = nn.CrossEntropyLoss()(data,author_labels)

        loss.backward()

        optimizer.step()
        loop.set_description(f"Epoch [{epoch + 1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item())


    print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")


mps


Epoch [1/3]: 100%|██████████| 78/78 [01:51<00:00,  1.43s/it, loss=7.77]


Epoch: 1, Loss: 7.7713847160339355


Epoch [2/3]:  12%|█▏        | 9/78 [00:12<01:37,  1.41s/it, loss=7.74]


KeyboardInterrupt: 